In [1]:
import geopy.distance as dst
import math
import random as r

In [2]:
def calcLength(coords_1, coords_2):
    return dst.geodesic(coords_1, coords_2).m

def writeNodes(path:str, nodes):
    line = ""
    for nd, nd_dt in enumerate(nodes):
        line += "\n" + str(nd) + " " + str(nd_dt[0]) + " " + str(nd_dt[1])
    file = open(path, "w+")
    file.write(line)
    file.close()

def writeAdjList(path:str, adj_lst):
    line = ""
    for nd, adj_nodes in enumerate(adj_lst):
        line += "\n" + str(nd)
        for adj_nd in adj_nodes:
            adj_nd_dt = adj_nodes[adj_nd]
            line += " " + str(adj_nd) + " " + str(adj_nd_dt[0]) + " " + str(adj_nd_dt[1])
    file = open(path, "w+")
    file.write(line)
    file.close()

def getDataNodes(ntx, path_nodes:str, path_adj_list:str):
    id2indx = {}
    n = len(ntx.nodes.data(True))
    nodes = []
    adj_lst = []
    for i, ntx_nd in enumerate(ntx.nodes.data(True)):
        id2indx[ntx_nd[0]] = i
        nodes.append((ntx_nd[1]['y'], ntx_nd[1]['x']))
        adj_lst.append({})
    for nd_id in ntx:
        nd = id2indx[nd_id]
        for adj_id in ntx[nd_id]:
            adj_nd = id2indx[adj_id]
            lngth = calcLength(nodes[nd], nodes[adj_nd])
            adj_lst[nd][adj_nd] = [lngth, r.randint(0, 100) / 100]
    writeNodes(path_nodes, nodes)
    writeAdjList(path_adj_list, adj_lst)

def only_path(root, end, path):
    if path[end] == -1:return []
    n_path = [end]
    while n_path[-1] != root:
        n_path.append(path[n_path[-1]])
    return list(reversed(n_path))

def getCoefTrafic(hora):
    trafic_o = 0.6
    trafic_base = math.e/(math.sqrt(2 * math.pi) * trafic_o)
    hora = hora % 12
    coef = -0.5 * ((hora - 7.)/ trafic_o / 2 )**2
    value = trafic_base ** coef
    return value

In [3]:
#!pip install tkintermapview
#!pip install pyperclip

In [4]:
class Graph:
    def __init__(self, nodes_path = "nodes.txt", edges_path = "edges.txt"):
        self.nodes = [] #lista de informacion de nodos nodes[0] informacion del nodo 0
        self.adj_lst = [] #lista de listas de adjacencias adj_lst[0][1] informacion de la arista 0->1
        if len(nodes_path) > 2 and len(edges_path) > 2:
            file = open(nodes_path, 'r')
            next(file)
            for line in file:
                words = line.split()
                self.nodes.append((float(words[1]), float(words[2])))
                self.adj_lst.append({})
            file.close()
            file = open(edges_path, 'r')
            next(file)
            for line in file:
                words = line.split()
                nd = int(words[0])
                for adj_nd in range(1, len(words), 3):
                    self.adj_lst[nd][int(words[adj_nd])] = ((float(words[adj_nd + 1]), float(words[adj_nd + 2])))
            file.close()
            self.n = len(self.nodes)
            print("nodes:", len(self.nodes))

    def __getitem__(self, node):
        return self.nodes[node] if node in range(len(self.nodes)) else (0,0)

    def edge(self, nd, adj_nd):
        return self.adj_lst[nd][adj_nd]

    def generateHeuristic(self, end):
        end_data = self.nodes[end]
        return [calcLength(nd_dt, end_data) for nd_dt in self.nodes]

    def getWayNodes(self, root, end, hour = 7):
        return list(only_path(root, end, self.aStar(root, end, hour)))

    def getWayCoords(self, root, end, hour = 7):
        nodes_way = self.getWayNodes(root, end, hour)
        n = len(nodes_way)
        for id in range(n):
            nodes_way[id] = self.nodes[nodes_way[id]]
        return nodes_way

    def normalizePositions(self, pi, pf):
        if pf[0] < pi[0]:
            pi[0], pf[0] = pf[0], pi[0]
        if pf[1] < pi[1]:
            pi[1], pf[1] = pf[1], pi[1]
        
    def getAreaNodes(self, pi, pf, nodes = None):
        area_nodes = []
        if nodes is None:
            nodes = range(len(self.nodes))
        self.normalizePositions(pi, pf)
        for nd in nodes:
            nd_data = self.nodes[nd]
            if nd_data[0] < pi[0] or nd_data[0] > pf[0] or nd_data[1] < pi[1] or nd_data[1] > pf[1]:
                continue
            area_nodes.append(nd)
        return area_nodes

    def getNearestNodes(self, pos, n):
        a_nodes = self.getAreaNodes([pos[0] - n, pos[1] - n], [pos[0] + n, pos[1] + n])
        if len(a_nodes) == 0: return []
        while True:
            n /= 2
            b_nodes = self.getAreaNodes([pos[0] - n, pos[1] - n], [pos[0] + n, pos[1] + n], a_nodes)
            new_len = len(b_nodes)
            if new_len > 10:
                a_nodes = b_nodes
            elif new_len > 0:
                return b_nodes
            else:
                return a_nodes

    def getNearestNode(self, pos, n = 0.001):
        nearest_nodes = self.getNearestNodes(pos, n)
        if len(nearest_nodes) == 0: return None
        nrst_nd = -1
        min = math.inf
        for nd in nearest_nodes:
            dist = calcLength(self.nodes[nd], pos)
            if dist < min:
                nrst_nd = nd
                min = dist
        return nrst_nd

    def getCoefTrafic(hora):
        trafic_o = 0.6 #dispersion
        trafic_base = math.e/(math.sqrt(2 * math.pi) * trafic_o)
        hora = hora % 12
        coef = -0.5 * ((hora - 7.)/ trafic_o / 2 )**2
        value = trafic_base ** coef
        return value

    def pathDistance(self, root, end, path):
        if path[end] == -1: 
            if root == end: 
                return 0
            return -1
        nd = end
        distance = 0
        while nd != root:
            distance += self.adj_lst[path[nd]][nd][0]
            nd = path[nd]
        return distance
        
    def aStar(self, root, end, hour = 7):
        path = [-1] * self.n
        g = [math.inf] * self.n
        h = self.generateHeuristic(end)
        f = [math.inf] * self.n
        hour_trfc = getCoefTrafic(hour)
        traffic = [{} for _ in range(self.n)]
        g[root] = 0
        test_nds = set()
        def assignLabels(nd):
            for adj in self.adj_lst[nd]:
                nd_trfc = traffic[nd]
                nd_trfc[adj] = nd_trfc.get(adj, round(1 + self.adj_lst[nd][adj][1] * hour_trfc, 4))
                _g = g[nd] + self.adj_lst[nd][adj][0] * nd_trfc[adj]
                if g[adj] > _g:
                    path[adj] = nd
                    g[adj] = _g
                    f[adj] = _g + h[adj]
                    if adj not in test_nds: test_nds.add(adj)
        def getNextNode():
            min = math.inf
            id = -1
            for tst_nd in test_nds:
                if f[tst_nd] < min:
                    min = f[tst_nd]
                    id = tst_nd
            return id
        tmp_nd = root
        test_nds.add(tmp_nd)
        while not(tmp_nd == end or tmp_nd == -1):
            test_nds.remove(tmp_nd)
            assignLabels(tmp_nd)
            tmp_nd = getNextNode()
        return path

In [5]:
import tkinter
import tkintermapview
from tkinter import ttk

root = tkinter.Tk()
root.pack_propagate(0)
root.wm_title("MyGPS")
root.geometry("1024x768")
root.configure(background='black')

G = Graph()
n_markers = 0
markers = [[None, None], [None, None]]
nodes = [None] * 2
paths = [None] * 2
coords_ttl_lbl = [ttk.Label(root, text="Coordenada Origen", background="black", foreground="white", font=('Roboto', 12, 'bold')),
                  ttk.Label(root, text="Coordenada Destino", background="black", foreground="white", font=('Roboto', 12, 'bold'))]
coords_in = [[tkinter.Entry(root, width=30) for _ in range(2)] for _ in range(2)]
coords_lbls = [[ttk.Label(root, text="Lat", background="black", foreground="white"), ttk.Label(root, text="Lon", background="black", foreground="white")] for _ in range(2)]

_hour_ = 7
hour_str = tkinter.StringVar(value="7")
hour_lbl = ttk.Label(root, text="Hora", background="black", foreground="white", font=('Roboto', 12, 'bold'))
hour_in = tkinter.Entry(root, textvariable=hour_str, width= 5)

map_widget1 = tkintermapview.TkinterMapView(root, width=400, height=400, corner_radius=0)
map_widget1.set_address("Santiago de Surco")
map_widget1.set_zoom(13)

map_widget2 = tkintermapview.TkinterMapView(root, width=400, height=400, corner_radius=0)
map_widget2.set_address("Santiago de Surco")
map_widget2.set_zoom(13)

#Functions
def draw_path():
    global _hour_, nodes
    if None in nodes:
        return
    tmp = _hour_
    try:
        _hour_ = int(hour_in.get()[:2])
    except:
        _hour_ = tmp
    print(_hour_)
    hour_in.delete(0, tkinter.END)
    hour_str.set(str(_hour_))
    if None not in paths:
        paths[0].delete()
        paths[1].delete()
    paths[0] = map_widget1.set_path(G.getWayCoords(nodes[0], nodes[1], hour=_hour_))
    paths[1] = map_widget2.set_path(G.getWayCoords(nodes[0], nodes[1], hour=_hour_))

def push_button_path(*args):
    for i in range(2):
        try:
            lat = float(coords_in[i][0].get())
            lon = float(coords_in[i][1].get())
            print(lat, lon)
            if lat != "" and lon != "":
                add_marker([lat, lon], i)
        except:
            lat = lon = 0
    draw_path()

def add_marker(coords, id):
    node = G.getNearestNode(coords, 0.001)
    if node is not None:
        nodes[id] = node
        if markers[0][id] is not None and markers[1][id] is not None:
            markers[0][id].delete()
            markers[1][id].delete()
        markers[0][id] = map_widget1.set_marker(coords[0], coords[1], text="Mark0" + str(id))
        markers[1][id] = map_widget2.set_marker(coords[0], coords[1], text="Mark1" + str(id))
        return True
    return False

def add_marker_event(coords):
    global markers, n_markers, _hour_
    print("Add marker:", coords)
    if n_markers < 2:
        if add_marker(coords, n_markers):
            draw_path()
            n_markers = (n_markers + 1) % 2

path_btn = tkinter.Button(text="CAMINO", command=push_button_path)
map_widget1.add_right_click_menu_command(label="Add Marker", command=add_marker_event, pass_coords=True)
map_widget2.add_right_click_menu_command(label="Add Marker", command=add_marker_event, pass_coords=True)

k = 0
count = 0
for i in range(4):
    n_coord = int(i/2)
    if i % 2 == 0:
        coords_ttl_lbl[n_coord].grid(row=i+k, column=1, pady=4, padx=1)
        k+=1
    coords_lbls[n_coord][i%2].grid(row=i+k, column=0, sticky = tkinter.W, pady=4, padx=1)
    coords_in[n_coord][i%2].grid(row=i+k, column=1, sticky = tkinter.W, pady=4, padx=1)
    count +=1
count += k


path_btn.grid(row=count, column=1, pady=4, padx=1)
count+=1
hour_lbl.grid(row=count, column=0, pady=4, padx=1)
hour_in.grid(row=count, column=1, pady=4, padx=1)
count+=1
#path_button.grid(row=count + 1, column=1, pady=4, padx=1)

map_widget1.place(x=600, y=10)
map_widget2.place(x=600, y=420)

root.mainloop()

nodes: 3888
Add marker: (-12.12431857768909, -76.98868362446706)
Add marker: (-12.137749539039545, -76.99795333882253)
7
12.22 13.22
7
-12.1304369 -76.982142
7
